In [43]:
import pandas as pd
import geopandas as gpd
from tabula import read_pdf
import googlemaps
import folium

## Data Links:

Cook County Data: https://datacatalog.cookcountyil.gov/Property-Taxation/Assessor-Neighborhood-Boundaries/pcdw-pxtg/about_data

McHenry Open Data: https://data-mchenrycountygis.opendata.arcgis.com/datasets/fe6a3cf3369d426ea7a4dd15b06aade6_0/explore?location=42.323836%2C-88.451676%2C10.56&showTable=true

Lake County Open Data: https://data-lakecountyil.opendata.arcgis.com/datasets/811621907d3649778ef35ec37731e921/explore

DuPage County Open Data: https://gisdata-dupage.opendata.arcgis.com/datasets/DuPage::municipalities/explore?location=41.839442%2C-88.087870%2C10.69

In [2]:
# cook_county_file_path = 'Assessor - Neighborhood Boundaries_20240510.geojson'
# cook_county_gdf = gpd.read_file(cook_county_file_path)

# mchenry_county_file_path = 'McHenry_Political_Townships.geojson'
# mchenry_county_gdf = gpd.read_file(mchenry_county_file_path)

# lake_county_file_path = 'Lake_County_Municipal_Boundaries.geojson'
# lake_county_gdf = gpd.read_file(lake_county_file_path)

# dupage_county_file_path = 'Du_Page_Municipalities.geojson'
# dupage_county_gdf = gpd.read_file(dupage_county_file_path)

/Users/afarence/opt/anaconda3/lib/python3.9/site-packages/geopandas/io/file.py:344: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")


In [17]:
# Path to your PDF file
file_path = 'AHPAA-Non-Exempt-Historical-List.pdf'

# Reading the table from the PDF
# 'pages' can be 'all' or a list of pages like '1,2,3' or '1-3'
df_list = read_pdf(file_path, pages='1-2', multiple_tables=True)

# Combining tables if multiple tables are found
df = pd.concat(df_list)

/Users/afarence/opt/anaconda3/lib/python3.9/site-packages/tabula/io.py:1057: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


In [18]:
df = df[df['2023'] == 'Exempt']

In [19]:
# cook_county_gdf['township_name'].unique()

In [20]:
# mchenry_county_gdf['TOWNSHIP'].unique()

In [21]:
# # Your GeoDataFrame is 'gdf' and the column is 'location_name'
# lake_county_gdf['NAME'] = lake_county_gdf['NAME'].str.replace(r'\b(village|of|city)\b', '', regex=True, case=False)

# # Trim any extra spaces left by the removal
# lake_county_gdf['NAME'] = lake_county_gdf['NAME'].str.strip()


In [22]:
# lake_county_gdf['NAME'].unique()

In [23]:
# dupage_county_gdf['CITY'].unique()

In [24]:
df = df.drop(columns=['2003','2013','2018'])

In [34]:
df['full_address'] = df['Local Government Name'] + ' ' + ' ' + df['County'] + ' Illinois'

In [35]:
df

,Local Government Name,County,2023,full_address
0,Algonquin,McHenry,Exempt,Algonquin McHenry Illinois
1,Bannockburn,Lake,Exempt,Bannockburn Lake Illinois
2,Barrington,Cook,Exempt,Barrington Cook Illinois
4,Bartlett,DuPage,Exempt,Bartlett DuPage Illinois
8,Cary,McHenry,Exempt,Cary McHenry Illinois
11,Elburn,Kane,Exempt,Elburn Kane Illinois
13,Flossmoor,Cook,Exempt,Flossmoor Cook Illinois
14,Fox River Grove,McHenry,Exempt,Fox River Grove McHenry Illinois
17,Gilberts,Kane,Exempt,Gilberts Kane Illinois
20,Green Oaks,Lake,Exempt,Green Oaks Lake Illinois


In [36]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [37]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['full_address'].apply(geocode)

In [38]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

In [50]:
# Create a map
m = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=8.5)

# Add markers with HTML formatted popups
for index, row in df.iterrows():
    popup_content = f'<div style="width: 130px; text-align: center;"><strong>{row["Local Government Name"]}, {row["County"]} County</strong></div>'
    folium.Marker(
        [row['lat'], row['lon']],
        popup=folium.Popup(popup_content, max_width=250)
    ).add_to(m)

# Display the map
m.save('index.html')
m  # this will display the map if you're in a Jupyter notebook environment